# Sampling and Characterization Peptides Protocol

First create the peptide structure and then run it in *sander* or *pmemd.cuda*. This can be done easily with the tleap program placing this information in a file *tleap.in*:

In [42]:
tleap_in = """source leaprc.ff14SB
peptide = sequence { NASP ARG VAL TYR ILE HIS CPRO }
addions peptide Na+ 0
addions peptide Cl- 0
solvateBox peptide TIP3PBOX 13
saveAmberParm peptide p.1.top p.1.crd
quit
"""

open('tleap.in', 'w').write(tleap_in)

192

In [33]:
!gsutil cp gs://sbglabdata/src/gpu/leaprc.ff14SB .
!gsutil cp gs://sbglabdata/src/gpu/min.1.in .
!gsutil cp gs://sbglabdata/src/gpu/heat.*.inp .
!gsutil cp gs://sbglabdata/src/gpu/dmd.1.inp .
!gsutil cp gs://sbglabdata/src/gpu/eq.1.inp .    

Copying gs://sbglabdata/src/gpu/leaprc.ff14SB...
/ [1 files][  5.2 KiB/  5.2 KiB]                                                
Operation completed over 1 objects/5.2 KiB.                                      
Copying gs://sbglabdata/src/gpu/min.1.in...
/ [1 files][   74.0 B/   74.0 B]                                                
Operation completed over 1 objects/74.0 B.                                       
Copying gs://sbglabdata/src/gpu/heat.1.inp...
Copying gs://sbglabdata/src/gpu/heat.2.inp...                                   
Copying gs://sbglabdata/src/gpu/heat.3.inp...                                   
/ [3 files][  831.0 B/  831.0 B]                                                
Operation completed over 3 objects/831.0 B.                                      
Copying gs://sbglabdata/src/gpu/dmd.1.inp...
/ [1 files][  199.0 B/  199.0 B]                                                
Operation completed over 1 objects/199.0 B.                                      
Co

In [31]:
!cat leaprc.ff14SB

logFile leap.log
#
# ----- leaprc for loading the ff14SB force field
# ----- NOTE: this is designed for PDB format 3!
#    Uses frcmod.ff14SB for proteins; ff99bsc0 for DNA; ff99bsc0_chiOL3 for RNA
#
#	load atom type hybridizations
#
addAtomTypes {
	{ "H"   "H" "sp3" }
	{ "HO"  "H" "sp3" }
	{ "HS"  "H" "sp3" }
	{ "H1"  "H" "sp3" }
	{ "H2"  "H" "sp3" }
	{ "H3"  "H" "sp3" }
	{ "H4"  "H" "sp3" }
	{ "H5"  "H" "sp3" }
	{ "HW"  "H" "sp3" }
	{ "HC"  "H" "sp3" }
	{ "HA"  "H" "sp3" }
	{ "HP"  "H" "sp3" }
	{ "HZ"  "H" "sp3" }
	{ "OH"  "O" "sp3" }
	{ "OS"  "O" "sp3" }
	{ "O"   "O" "sp2" }
	{ "O2"  "O" "sp2" }
	{ "OP"  "O" "sp2" }
	{ "OW"  "O" "sp3" }
	{ "CT"  "C" "sp3" }
	{ "CX"  "C" "sp3" }
	{ "C8"  "C" "sp3" }
	{ "2C"  "C" "sp3" }
	{ "3C"  "C" "sp3" }
	{ "CH"  "C" "sp3" }
	{ "CS"  "C" "sp2" }
	{ "C"   "C" "sp2" }
	{ "CO"   "C" "sp2" }
	{ "C*"  "C" "sp2" }
	{ "CA"  "C" "sp2" }
	{ "CB"  "C" "sp2" }
	{ "CC"  "C" "sp2" }
	{ "CN"  "C" "sp2" }
	{ "CM"  "C" "sp2" }
	{ "CK"  "C" "sp2" }
	{ "CQ"  "C" "s

## Step 1

La secuencia de residuo tiene los nombres con 3 letras identificatorios de cada amino ́acidos, excepto el 1ro y el  ́ultimo a los cuales se le agrega la N y la C, respectivamente. 

En este caso el tleap se usa para generar el modelo. Esto generara dos archivos. Uno
de topolog ́ıa (p.1.top) y otro de coordenadas (p.1.crd). De aqu ́ı en adelante siempre
tendremos dos archivos para correr las dinámicas. Uno es el de topolog ́ıa y siempre
ser ́a el mismo, mientras que el otro es el que tiene las coordenadas del sistema y va
a ser variable en cada paso.

Es necesario correr el programa mediante la lınea:

In [43]:
!~/amber20/bin/tleap -s -f tleap.in

-I: Adding /home/sebastian/amber20/dat/leap/prep to search path.
-I: Adding /home/sebastian/amber20/dat/leap/lib to search path.
-I: Adding /home/sebastian/amber20/dat/leap/parm to search path.
-I: Adding /home/sebastian/amber20/dat/leap/cmd to search path.
-s: Ignoring startup file: leaprc
-f: Source tleap.in.

Welcome to LEaP!
Sourcing: ./tleap.in
----- Source: ./leaprc.ff14SB
----- Source of ./leaprc.ff14SB done
Log file: ./leap.log
Loading parameters: /home/sebastian/amber20/dat/leap/parm/parm10.dat
Reading title:
PARM99 + frcmod.ff99SB + frcmod.parmbsc0 + OL3 for RNA
Loading parameters: /home/sebastian/amber20/dat/leap/parm/frcmod.ff14SB
Reading force field modification type file (frcmod)
Reading title:
ff14SB protein backbone and sidechain parameters
Loading library: /home/sebastian/amber20/dat/leap/lib/amino12.lib
Loading library: /home/sebastian/amber20/dat/leap/lib/aminoct12.lib
Loading library: /home/sebastian/amber20/dat/leap/lib/aminont12.lib
Loading library: /home/sebastia

## Step 2

**Minimizar**: De aca en adelante solo se utilizar ́a el comando *sander o pmemd.cuda* del Amber. Para
minimizar debemos tener 3 archivos de *inputs*: Uno de topolog ́ıa (p.1.top) ,otro de
coordenadas (p.1.crd) y el que le indicamos al programa las variables a realizar. Este
es el (min.1.in) y se escribe a continuación

In [34]:
!cat min.1.in

Minimizacion
&cntrl
imin=1,
maxcyc=8000,
ncyc=7500,
cut=9,
ntpr=200,
&end


Se puede minimizar ejecutando el comando así:

In [44]:
!time ~/amber20/bin/pmemd.cuda -O -i min.1.in -o min.1.out -p p.1.top -c p.1.crd -r min.1.crd

Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL

real	0m6.807s
user	0m6.392s
sys	0m0.384s


El archivo .in es el *input*, el .out, el *output*. El .top el de topología generado con *tleap*, el p.1.crd es el archivo de coordenadas generado por el *tleap*. El min.1.crd es el archivo de coordenadas de salida que se utilizará, junto con el p.1.top en el próximo paso.

## Step 3

**Calentar**: Utilizar este *inputs* (lo llamaremos heat.1.inp) para que el sistema pase de 0 K a 100 K

In [39]:
!cat heat.1.inp


35 heating
&cntrl
imin=0,irest=0,ntx=1,ntb=1,cut=9.0,ntp=0,nmropt=1,
ntt=3,nstlim=10000,ntpr=500,ntwx=10000,dt=0.002,ioutfm=1,
ntwr=10000,gamma_ln=1.0,tol=0.0000001,ntf=2,ntc=2,
&end
&wt
type="TEMP0", istep1=0, istep2=10000, value1=0.0, value2=100.0,
&end
&wt
type="END",
&end


Luego hay que correrlo ejecutando el siguiente comando:

In [45]:
!time ~/amber20/bin/pmemd.cuda -O -i heat.1.inp -o heat.1.out -p p.1.top -c min.1.crd -r heat.1.crd -x heat.1.traj

Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL

real	0m5.148s
user	0m4.705s
sys	0m0.400s


Esta corrida dará como resultado el archivo heat.1.crd que tiene las posiciones y las
velocidades correspondientes a un sistema a 100 K.
Utilizar este *input* (lo llamaremos heat.2.inp) para que el sistema pase de 100 K a
200 K

In [41]:
!cat heat.2.inp

35 heating
&cntrl
imin=0,irest=1,ntx=5,ntb=2,cut=9.0,ntp=2,nmropt=1,
ntt=3,nstlim=5000,ntpr=500,ntwx=5000,dt=0.002,ioutfm=1,
ntwr=5000,gamma_ln=1.0,tol=0.0000001,ntf=2,ntc=2,
&end
&wt
type="TEMP0", istep1=0, istep2=5000, value1=100.0, value2=200.0,
&end
&wt
type="END",
&end


Luego hay que correrlo ejecutando el siguiente comando:

In [46]:
!time ~/amber20/bin/pmemd.cuda -O -i heat.2.inp -o heat.2.out -p p.1.top -c heat.1.crd -r heat.2.crd -x heat.2.traj

Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL

real	0m4.319s
user	0m3.969s
sys	0m0.324s


Esta corrida dará como resultado el archivo heat.2.crd que tiene las posiciones y las velocidades correspondientes a un sistema a 200 K.
Utilizar este *input* (lo llamaremos heat.3.inp) para que el sistema pase de 200 K a 300 K

In [10]:
!cat heat.3.inp

35 heating
&cntrl
imin=0,irest=1,ntx=5,ntb=2,cut=9.0,ntp=2,nmropt=1,
ntt=3,nstlim=10000,ntpr=500,ntwx=10000,dt=0.002,ioutfm=1,
ntwr=10000,gamma_ln=1.0,tol=0.0000001,ntf=2,ntc=2,
&end
&wt
type="TEMP0", istep1=0, istep2=10000, value1=200.0, value2=300.0,
&end
&wt
type="END",
&end


Luego hay que correrlo ejecutando el siguiente comando:

In [47]:
!time ~/amber20/bin/pmemd.cuda -O -i heat.3.inp -o heat.3.out -p p.1.top -c heat.2.crd -r heat.3.crd -x heat.3.traj

Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL

real	0m8.215s
user	0m7.834s
sys	0m0.356s


## Step 4 

**Equilibración**: Utilizar este *inputs* (lo llamaremos eq.1.inp) para que el sistema acomode su densidad.  

In [50]:
!cat eq.1.inp

Luego hay que correrlo ejecutando el siguiente comando:

In [48]:
!time ~/amber20/bin/pmemd.cuda -O -i eq.1.inp -o eq.1.out -p p.1.top -c heat.3.crd -r eq.1.crd -x eq.1.traj

Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL

real	0m15.059s
user	0m14.626s
sys	0m0.408s


Esta corrida dará como resultado el archivo eq.1.crd que tiene las posiciones y las velocidades correspondientes a un sistema a 303 K con la densidad equilibrada.
Utilizaremos este archivo y el .top para correr la producción.

## Step 5

**Producción**: Con el siguiente inputs (dmd.1.inp) se corren las simulaciones a 303 K utilizando el pmemd.cuda que funciona en GPU. Este archivo se será así:

In [51]:
!cat dmd.1.inp

Luego es necesario correrlo con el siguiente comando:

In [49]:
!time ~/amber20/bin/pmemd.cuda -O -i dmd.1.inp -o dmd.1.out -p p.1.top -c heat.3.crd -r prod.1.crd -x dmd.1.traj

Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL

real	12m2.397s
user	11m57.490s
sys	0m5.020s


## Step 6

**Análisis**: Analizar el conjunto de estructuras.